In [20]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#creation of dataset
import lib.ml_workflow.create_dataset as cds
from lib.raman_lib.misc import load_data

#quality control
import lib.ml_workflow.quality_control as qc
from lib.raman_lib.preprocessing import RangeLimiter
from lib.raman_lib.visualization import plot_spectra_peaks
from lib.raman_lib.spectra_scoring import score_names

#preprocessing
from lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import GroupKFold

#file handling
from pathlib import Path
import os

In [2]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [3]:
FILE_PREFIX = "HL_428_E_R"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR+"/"+FILE_PREFIX+"_qc.csv"
PREP_OUT= RESULT_DIR+"/"+FILE_PREFIX+"_preprocessed.csv"


## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [4]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [5]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [6]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [7]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [8]:
PREP_LIM_LOW=QC_LIM_LOW
PREP_LIM_HIGH=QC_LIM_HIGH

### Window-width for smoothing

In [9]:
PREP_WINDOW=15

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [10]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_etoposide, path_resveratrol], ['Etoposide', 'Resveratrol'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [11]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:

    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [22]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [23]:
print(PREP_OUT)
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
group_kfold = GroupKFold(n_splits=3)

for i, (train_index, test_index) in enumerate(group_kfold.split(X, y, groups)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}, group={groups[train_index]}")
    print(f"  Test:  index={test_index}, group={groups[test_index]}")

./result/HL_428_E_R/HL_428_E_R_preprocessed.csv
Fold 0:
  Train: index=[  1   2   6   7   8   9  10  12  13  14  15  16  17  18  21  22  23  24
  25  26  28  29  31  32  36  37  39  40  41  42  46  47  48  50  51  52
  53  54  55  58  59  61  62  63  64  65  67  69  70  71  73  74  75  76
  77  78  79  81  82  83  85  86  87  88  89  92  93  94  95  96  97  98
  99 101 102 103 106 107 108 109 110 111 112 115 116 117 120 121 122 123
 125 126 127 128 129 130 134 135 137 138 139 140 141 142 143 144 145 146
 147 148 149 152 154 157 158 159 160 162 163 164 165 166 167 168 169 171
 172 177 178 180 182 183 184 185 186 190 191 192 193 194 195 196 198 199
 200 201 203 205 207 209 210 211 212 216 217 219 222 224 225 226 227 229
 231 232 233 234 236 237 240 242 244 245 246 247 248 249 252 255 257 258
 259 260 261 265 266 267 270 271 272 274 276 278 281 282 283 284 285 287
 288 289 293 294 295 296 297 298 302 303 304 307 310 312 313 314 315 316
 321 322 323 324 325 328 329 331 332 333 334 335 336 